In [ ]:
import scipy.stats
import discohisto
import discohisto.stats
import lebesgue
import numpy
from matplotlib import pyplot

In [ ]:
pyplot.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.size": 10,
        "figure.facecolor": "w",
    }
)

In [ ]:
def plot_poisson(n):
    x = numpy.linspace(0, 10, 100)
    y = discohisto.stats.poisson_log_minus_max(n, x)
    pyplot.plot(x, y, "k")
    pyplot.ylim(-8, 0.5)
    pyplot.show()
    
plot_poisson(0)
plot_poisson(1)

In [ ]:
# https://ui.adsabs.harvard.edu/abs/1998PhRvD..57.3873F/abstract
# Table VI
FENDMAN_COUSINS_POISSON_95 = [
    (0.00, 3.09),
    (0.05, 5.14), 
    (0.36, 6.72),
    (0.82, 8.25),
    (1.37, 9.76),
    (1.84, 11.26),
    (2.21,12.75),
    (2.58,13.81),
    (2.94,15.29),
    (4.36,16.77),
    (4.75,17.82),
    (5.14,19.29),
    (6.32,20.34),
    (6.72,21.80),
    (7.84,22.94),
    (8.25,24.31),
    (9.34,25.40),
    (9.76,26.81),
    (10.84,27.84),
    (11.26,29.31),
    (12.33,30.33),
]


def feldman_cousins_poisson_95(n):
    assert n == int(n)
    return FENDMAN_COUSINS_POISSON_95[int(n)]

In [ ]:
def poisson_uniform_prior_central_interval(n, level):
    posterior = scipy.stats.gamma(n + 1)
    return posterior.ppf([0.5 * level, 1 - 0.5 * level])

In [ ]:
def plot_poisson_interval(n):
    figure, axis = pyplot.subplots(
        dpi=400,
        figsize=numpy.array([4, 3]) * 0.7,
        gridspec_kw={
            "top": 0.99,
            "right": 0.99,
            "bottom": 0.2,
            "left": 0.18,
        },
    )
    
    level = 0.05
    
    _, xhi = lebesgue.likelihood.poisson(n).interval(numpy.exp(-8))
    
    x = numpy.linspace(0, xhi, 511)
    y = discohisto.stats.poisson_log_minus_max(n, x)
    axis.plot(x, y, "k")
    
    # likelihood interval (yes e**-3 != 0.05, but it's close enough)
    lo, hi = lebesgue.likelihood.poisson(n).interval(numpy.exp(-3))
    three = axis.axvline(lo, color="xkcd:blue")
    axis.axvline(hi, color="xkcd:blue")
    
    # another at -2
    lo, hi = lebesgue.likelihood.poisson(n).interval(numpy.exp(-2))
    two = axis.axvline(lo, ls=":", color="xkcd:blue")
    axis.axvline(hi, ls=":", color="xkcd:blue")
    
    # Feldman-Cousins (constrained maximum likelihood ratio test statistic)
    lo, hi = feldman_cousins_poisson_95(n)
    feldman_cousins, = axis.plot([lo, hi], [-5] * 2, "xkcd:red")
    
    # Posterior with improper uniform prior
    lo, hi = poisson_uniform_prior_central_interval(n, level)
    print(lo, hi)
    credible_interval, = axis.plot([lo, hi], [-6] * 2, "xkcd:barney purple")
    
    axis.text(
        0.98,
        0.98,
        r"$n=%d$" % n,
        horizontalalignment="right",
        verticalalignment="top",
        transform=axis.transAxes,
    )
    
    axis.set_ylim(-8, 0.5)
#     axis.set_xlim(0, xhi * 2)
    axis.set_xlabel(r"$\mu$")
    axis.set_ylabel(r"$\log L(\mu) - \log L(\check{\mu})$")
    
#     axis.legend(
#         [two, three, feldman_cousins, credible_interval],
#         [r"$\log R = -2$", r"$\log R = -3$", "Feldman-Cousins", "Credible Interval"],
#         frameon=False,
#         loc="upper right",
#     )
    
    figure.savefig("intervals_%d.png" % n)
    pyplot.show()
    pyplot.close(figure)
    
plot_poisson_interval(0)
plot_poisson_interval(10)

In [ ]:
# green yellow orange red tasty colours yo